<a href="https://colab.research.google.com/github/tomhyhan/ML/blob/main/ML/colab/food.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch as t
import torchvision
import torchvision.transforms as tt
from torch.utils.data import Subset, random_split
from torch.utils.data.dataloader import DataLoader
from torch import nn


In [ ]:
def load_data():
    transform = tt.Compose([
        tt.Resize(256),
        tt.CenterCrop(224),
        tt.ToTensor(),
        tt.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    food101_data =torchvision.datasets.Food101(download=True, root="./", transform=transform)
    print("transform done")
    subset_indices = set()
    subset_images = []
    for i in range(len(food101_data)):
        if i % 1000 == 0:
            print("iter: ", i)
            print(subset_indices)
        subset_indices.add(food101_data[i][1])
        subset_images.append(i)
        if len(subset_indices) > 20:
            break
    food101_subset = Subset(food101_data, subset_images)
    print("len: ", len(food101_subset))
    traning_i = int(len(food101_subset) * 0.9)
    test_i = len(food101_subset) - traning_i
    f_train, f_test = random_split(food101_subset, [traning_i, test_i])
    return f_train, f_test

f_train, f_test = load_data()
print(len(f_train), len(f_test))

transform done
iter:  0
set()
iter:  1000
{54, 23}
iter:  2000
{85, 54, 23}
iter:  3000
{86, 85, 54, 23}
iter:  4000
{77, 23, 85, 86, 54, 92}
iter:  5000
{73, 77, 23, 85, 86, 54, 92}
iter:  6000
{4, 73, 77, 23, 85, 86, 54, 92}
iter:  7000
{4, 39, 73, 77, 48, 23, 85, 86, 54, 92}
iter:  8000
{96, 4, 39, 73, 77, 48, 23, 85, 86, 54, 92}
iter:  9000
{96, 4, 70, 39, 73, 77, 48, 23, 85, 86, 54, 92}
iter:  10000
{96, 4, 70, 39, 73, 77, 78, 48, 81, 23, 85, 86, 54, 92}
iter:  11000
{96, 4, 70, 39, 73, 77, 78, 48, 81, 80, 23, 85, 86, 54, 92}
iter:  12000
{96, 4, 70, 39, 7, 73, 77, 78, 48, 81, 80, 23, 85, 86, 54, 92}
iter:  13000
{96, 0, 4, 70, 39, 7, 73, 6, 77, 78, 48, 81, 80, 23, 85, 86, 54, 92}
iter:  14000
{0, 4, 6, 7, 23, 26, 39, 48, 54, 70, 73, 77, 78, 80, 81, 85, 86, 92, 96}
iter:  15000
{0, 4, 6, 7, 23, 26, 39, 48, 54, 70, 73, 77, 78, 80, 81, 84, 85, 86, 92, 96}
len:  15001
13500 1501


In [ ]:
# parameters
learning_rate = 0.001
training_epochs = 5
batch_size = 10

In [ ]:
class CNN(nn.Module):
    def __init__(self):
      super(CNN, self).init()
      self.build_net()

    def build_net(self):
        self.dropout = 0.5
        # Sizes
        # image: (batch_size, 3, 224, 224)
        # Conv: (batch_size, 32, 220, 220)
        # Pool: (batch_size, 32, 110, 110)
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        # Sizes
        # image: (batch_size, 32, 109, 109)
        # Conv: (batch_size, 64, 105, 105)
        # Pool: (batch_size, 64, 52, 52)
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        # Sizes
        # image: (batch_size, 64, 52, 52)
        # Conv: (batch_size, 128, 48, 48)
        # Pool: (batch_size, 128, 24, 24)
        self.conv3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc1 = nn.Sequential(
            nn.Linear(128* 24 * 24, 512),
            nn.ReLU(),
            nn.Dropout(self.dropout)
        )
        self.fc2 = nn.Linear(512, 20)

        self.cost_fn = nn.CrossEntropyLoss()
        self.optimizer = t.optim.Adam(self.parameters(), lr=learning_rate)

    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(x)
        out = self.conv3(x)
        out = self.view(out.size(0), -1)
        out = self.fc1(x)
        out = self.fc2(x)
        return out

    def predict(self, x):
        self.eval()
        return self.forward(x)

In [ ]:
t.cuda.is_available()
print(len(f_train))
print(f_train[0][0].shape)

13500
torch.Size([3, 224, 224])


In [ ]:
t = t.randn(20, 16, 50, 32)
print(t.size())

AttributeError: 'Tensor' object has no attribute 'randn'

In [ ]:
batch_size = 10

train_loader = DataLoader(f_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(f_test, batch_size=len(f_test), shuffle=False)

13500
